# Create consensus peaks with Signac

# Set-up

In [ ]:
# Imports
library(rtracklayer)
library(GenomicRanges)

In [ ]:
# Directory with called peaks
peaks_dir <- "/cellar/users/aklie/data/igvf/beta_cell_networks/peaks/igvf_sc-islet_10X-Multiome/25Aug23"

In [ ]:
# TODO: List out all the narrow peak files
peak_files <- NULL

In [ ]:
for (i in seq_along(along.with = unique.groups)) {
    narrowPeakPath <- paste0(
        outdir,
        .Platform$file.sep,
        unique.groups[[i]],
        "_peaks.narrowPeak"
    )
    print(narrowPeakPath)
    df <- read.table(
        file = narrowPeakPath,
        col.names = c(
            "chr", "start", "end", "name",
            "score", "strand", "fold_change",
            "neg_log10pvalue_summit", "neg_log10qvalue_summit",
            "relative_summit_position"
        )
    )
    gr <- makeGRangesFromDataFrame(df = df, keep.extra.columns = TRUE, starts.in.df.are.0based = TRUE)
    grlist[[i]] <- gr
    if (length(x = gr) > 0) {
        gr$ident <- unique.groups[[i]]
        grlist[[i]] <- gr
      } else {
        message("No peaks found for ", unique.groups[[i]])
    }
}

In [ ]:
# combine peaks and reduce, maintaining ident information
gr.combined <- Reduce(f = c, x = grlist)
gr <- reduce(x = gr.combined, with.revmap = TRUE)
dset.vec <- vector(mode = "character", length = length(x = gr))
ident.vec <- gr.combined$ident
revmap <- gr$revmap
for (i in seq_len(length.out = length(x = gr))) {
    datasets <- ident.vec[revmap[[i]]]
    dset.vec[[i]] <- paste(unique(x = datasets), collapse = ",")
}
gr$peak_called_in <- dset.vec
gr$revmap <- NULL

In [ ]:
export.bed(gr, con=file.path(peaks_dir, "consensus_peaks.bed"))

# reload